In [1]:
from lightgbm import LGBMClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold,train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

In [2]:
app_train = pd.read_csv('application_train_imputed.csv')
print(app_train.shape)
app_train['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
app_train['DAYS_EMPLOYED_PERC'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']
app_train['INCOME_CREDIT_PERC'] = app_train['AMT_INCOME_TOTAL'] / app_train['AMT_CREDIT']
app_train['INCOME_PER_PERSON'] = app_train['AMT_INCOME_TOTAL'] / app_train['CNT_FAM_MEMBERS']
app_train['ANNUITY_INCOME_PERC'] = app_train['AMT_ANNUITY'] / app_train['AMT_INCOME_TOTAL']
app_train['PAYMENT_RATE'] = app_train['AMT_ANNUITY'] / app_train['AMT_CREDIT']

print(app_train.shape)


(307511, 247)
(307511, 252)


In [3]:
y = app_train['TARGET']
X=app_train.drop(['TARGET'],axis=1)
# X.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
# Remove above line of Unnamed: 0 after making changes to application_train_imputed.csv

# Imputed csv + self added 5 cols 

In [5]:
X_train_later, X_test, y_train_later, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)
print("Postive examples in train set: {}".format(np.sum(y_train_later==0)))
print("Negative examples in train set: {}".format(np.sum(y_train_later==1)))

print("Postive examples in test set: {}".format(np.sum(y_test==0)))
print("Negative examples in test set: {}".format(np.sum(y_test==1)))

Postive examples in train set: 226148
Negative examples in train set: 19860
Postive examples in test set: 56538
Negative examples in test set: 4965


In [35]:
# X_train, X_val, y_train, y_val = train_test_split(X_train_later, y_train_later, test_size=0.2, stratify=y_train, random_state=1)


In [6]:
clf11 = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

In [7]:
clf11.fit(X_train_later,y_train_later)
y_pred=clf11.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
newdf=pd.DataFrame(data=y_pred)
newdf.iloc[:,0].value_counts()

0.0    61012
1.0      491
Name: 0, dtype: int64

In [9]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.92      1.00      0.96     56538
        1.0       0.46      0.05      0.08      4965

avg / total       0.89      0.92      0.89     61503



In [10]:
confusion_matrix(y_test, y_pred)

array([[56273,   265],
       [ 4739,   226]], dtype=int64)

# With SMOTE + above all

In [14]:
X_train_later, X_test, y_train_later, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)
print("Postive examples in train set: {}".format(np.sum(y_train_later==0)))
print("Negative examples in train set: {}".format(np.sum(y_train_later==1)))

print("Postive examples in test set: {}".format(np.sum(y_test==0)))
print("Negative examples in test set: {}".format(np.sum(y_test==1)))

sm = SMOTE(random_state=12, ratio = 1.0)
X_train_res, y_train_res = sm.fit_sample(X_train_later, y_train_later)

Postive examples in train set: 226148
Negative examples in train set: 19860
Postive examples in test set: 56538
Negative examples in test set: 4965


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [21]:
clf12 = LGBMClassifier(
            nthread=4,
            n_estimators=100,
            learning_rate=0.2,
            num_leaves=300,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

### After smote size of train set : 

In [22]:
print("Postive examples in train set: {}".format(np.sum(y_train_res==0)))
print("Negative examples in train set: {}".format(np.sum(y_train_res==1)))


Postive examples in train set: 226148
Negative examples in train set: 226148


In [23]:
clf12.fit(X_train_res,y_train_res)
y_pred=clf12.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [24]:
newdf=pd.DataFrame(data=y_pred)
newdf.iloc[:,0].value_counts()

0.0    61174
1.0      329
Name: 0, dtype: int64

In [25]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.92      1.00      0.96     56538
        1.0       0.52      0.03      0.06      4965

avg / total       0.89      0.92      0.89     61503



In [26]:
confusion_matrix(y_test, y_pred)

array([[56379,   159],
       [ 4795,   170]], dtype=int64)